### Imports etc

In [1]:
import sys
from pathlib import Path
import pandas as pd
import seaborn as sns

REPO_PATH = Path.home() / "workspace/jetpointnet"
SCRIPT_PATH = REPO_PATH / "python_scripts"
sys.path.append(str(SCRIPT_PATH))

import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ''
import tf
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# from mpl_toolkits.axes_grid1 import make_axes_locatable
from jets_training.models.JetPointNet import PointNetSegmentation
from jets_training.jets_visualize import generate_images_and_metadata
from jets_training.jets_train import (
    TRAIN_INPUTS,
    MAX_SAMPLE_LENGTH,
    baseline_configuration,
    EXPERIMENT_NAME,
)
OUTPUT_ACTIVATION_FUNCTION = baseline_configuration['OUTPUT_ACTIVATION_FUNCTION']




2024-07-09 20:14:27.548569: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-09 20:14:28.803892: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9373] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-09 20:14:28.806400: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-09 20:14:29.090035: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1534] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-09 20:14:29.599783: I tensorflow/core/platform/cpu_feature_guar

Logged in as jhimmens


2024-07-09 20:14:44.451888: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1926] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46339 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:1b:00.0, compute capability: 8.6
2024-07-09 20:14:44.490295: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1926] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 1290 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:1c:00.0, compute capability: 8.6
2024-07-09 20:14:44.510288: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1926] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 4410 MB memory:  -> device: 2, name: NVIDIA RTX A6000, pci bus id: 0000:4f:00.0, compute capability: 8.6
2024-07-09 20:14:44.530244: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1926] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 1546 MB memory:  -> device: 3, name: NVIDIA RTX A6000, pci bus id: 0000:50:00.0, com

# Generate Event Images

In [ ]:
model_path = "/home/jhimmens/workspace/jetpointnet/models/rev_2/collected_data/PointNet_best_name=rosy-wood-862.keras"
sets_to_visualize = ["JZ2", "JZ3", "JZ4"]
max_file_per_set = 2
max_images_per_set = 10

MODEL_NAME = model_path.split('/')[-1]
VISUAL_PATH = REPO_PATH / "visualizations" / EXPERIMENT_NAME / MODEL_NAME

model = PointNetSegmentation(MAX_SAMPLE_LENGTH, 
                             num_features=len(TRAIN_INPUTS), 
                             num_classes=1, 
                             output_activation_function=OUTPUT_ACTIVATION_FUNCTION,
                             model_version=1)

model.load_weights(model_path)

generate_images_and_metadata(sets_to_visualize, model, max_file_per_set, max_images_per_set, VISUAL_PATH)